<a href="https://colab.research.google.com/github/NiklasTR/sirt3/blob/main/notebooks/diffdock.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DiffDock
Dock a PDB files and a SMILES with [DiffDock](https://github.com/gcorso/DiffDock).

Select Runtime / Run all to run an example PDB file and SMILES.

v2 improvements:
- works on proteins >1000 aas (with flag added to extract.py)
- works on standard GPU (by reducing batch size from 10 to 6)
- works with AlphaFold ids (AF-...) as well as PDB ids
- works with comma-delimited PDB_ids and/or SMILES
- runs smina to generate affinities (as DiffDock posed, or with smina minimization)
- shows results in a py3DMol view

colab by [@btnaughton](https://twitter.com/btnaughton)

In [ ]:
#@title PDB + SMILES input

PDB_id = '6ej2' #@param {type:"string"}
SMILES_or_pubchem_id = 'CCCCNC(=O)[C@H](C)C[C@H](O)[C@@H]2C[C@H](C)CCCCCN(CC)C(=O)c1cc(cc(c1)C(=O)N2)OCC' #@param {type:"string"}

#@markdown Download a tar file containing all results?
download_results = True #@param {type:"boolean"}

In [ ]:
import os
import requests
import time
from random import random

def download_pdb_file(pdb_id: str) -> str:
    """Download pdb file as a string from rcsb.org"""
    PDB_DIR ="/tmp/pdb/"
    os.makedirs(PDB_DIR, exist_ok=True)

    # url or pdb_id
    if pdb_id.startswith('http'):
        url = pdb_id
        filename = url.split('/')[-1]
    elif pdb_id.endswith(".pdb"):
        return pdb_id
    else:
        if pdb_id.startswith("AF"):
            url = f"https://alphafold.ebi.ac.uk/files/{pdb_id}-model_v3.pdb"
        else:
            url = f"http://files.rcsb.org/view/{pdb_id}.pdb"
        filename = f'{pdb_id}.pdb'

    cache_path = os.path.join(PDB_DIR, filename)
    if os.path.exists(cache_path):
        return cache_path

    pdb_req = requests.get(url)
    pdb_req.raise_for_status()
    open(cache_path, 'w').write(pdb_req.text)
    return cache_path

def download_smiles_str(pubchem_id: str, retries:int = 2) -> str:
    """Given a pubchem id, get a smiles string"""
    while True:
        req = requests.get(f"https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/CID/{pubchem_id}/property/CanonicalSMILES/CSV")
        smiles_url_csv = req.text if req.status_code == 200 else None
        if smiles_url_csv is not None:
            break
        if retries == 0:
            return None
        time.sleep(1+random())
        retries -= 1

    return smiles_url_csv.splitlines()[1].split(',')[1].strip('"').strip("'") if smiles_url_csv is not None else None

In [ ]:
if not PDB_id or not SMILES_or_pubchem_id:
    PDB_id = "6agt"
    SMILES_or_pubchem_id = "COc(cc1)ccc1C#N"
    print(f"No input supplied. Using example data: {PDB_id} and {SMILES_or_pubchem_id}")

# to run many PDB+smiles at once, fill in a list of PDB_files and smiles here...
pdb_files = [download_pdb_file(_PDB_id) for _PDB_id in PDB_id.split(",")]
smiless = [download_smiles_str(_SMILES_or_pubchem_id) if str(_SMILES_or_pubchem_id).isnumeric() else _SMILES_or_pubchem_id
           for _SMILES_or_pubchem_id in SMILES_or_pubchem_id.split(',') ]

with open("/tmp/input_protein_ligand.csv", 'w') as out:
    out.write("protein_path,ligand\n")
    for pdb_file in pdb_files:
        for smiles in smiless:
            out.write(f"{pdb_file},{smiles}\n")

## Importing protein-ligand pairs for affinity ranking

In [ ]:
%cd /
!git clone https://github.com/labdao/sirt3.git
%cd /sirt3
!git pull
!git submodule init
!git submodule update
%cd /sirt3/lib/diffdock
!git submodule init
!git submodule update
!pwd

/
Cloning into 'sirt3'...
remote: Enumerating objects: 6981, done.
remote: Counting objects: 100% (6645/6645), done.
remote: Compressing objects: 100% (483/483), done.
remote: Total 6981 (delta 6160), reused 6638 (delta 6157), pack-reused 336
Receiving objects: 100% (6981/6981), 60.47 MiB | 34.94 MiB/s, done.
Resolving deltas: 100% (6293/6293), done.
/sirt3
Already up to date.
Submodule 'lib/diffdock' (https://github.com/labdao-apps/diffdock.git) registered for path 'lib/diffdock'
Submodule 'lib/equibind' (https://github.com/labdao-apps/equibind.git) registered for path 'lib/equibind'
Cloning into '/sirt3/lib/diffdock'...
Cloning into '/sirt3/lib/equibind'...
Submodule path 'lib/diffdock': checked out '04c40637ccd5673ad886ce81ae9b97ab46737345'
Submodule path 'lib/equibind': checked out '3df09f851b575edda90bbf655036605f82851bca'
/sirt3/lib/diffdock
Submodule 'esm' (https://github.com/facebookresearch/esm.git) registered for path 'esm'
Cloning into '/sirt3/lib/diffdock/esm'...
Submodule 

In [ ]:
# processing the BACE1 reference set for local processing
# TODO replace with complete counterscreen set
import pandas as pd
df = pd.read_csv("/sirt3/data/external/d3r_benchmark/BACE1/BACE_score_compounds_D3R_GC4_answers.csv")
df["protein_path"] = "/tmp/pdb/6ej2.pdb"
df["ligand"] = df['SMILES']
df = df[["protein_path", "ligand"]]
df.to_csv("/sirt3/data/processed/affinity_ranking/BACE1/input_protein_ligand.csv")
df

,protein_path,ligand
0,/tmp/pdb/6ej2.pdb,CCCCNC(=O)[C@H](C)C[C@H](O)[C@@H]2C[C@H](C)CCC...
1,/tmp/pdb/6ej2.pdb,C[C@@H]1CCCCCCOCC(=O)N(C)[C@@H](C)C(=O)N[C@@H]...
2,/tmp/pdb/6ej2.pdb,CCCC2=CC(CNC[C@@H](O)[C@@H]1C[C@H](C)CCCCCCOCC...
3,/tmp/pdb/6ej2.pdb,CCN2CCCCC[C@@H](C)C[C@H](NC(=O)c1cc(ccc1)C2=O)...
4,/tmp/pdb/6ej2.pdb,CC(C)c5cc(CNC[C@@H](O)[C@@H]4C[C@H](C)CCCCCN([...
...,...,...
149,/tmp/pdb/6ej2.pdb,CCCCNC(=O)[C@H](C)C[C@H](O)[C@@H]2Cc3cc(CCCCCN...
150,/tmp/pdb/6ej2.pdb,CCCCNC(=O)[C@H](C)C[C@H](O)[C@@H]2Cc3cccc(CCCC...
151,/tmp/pdb/6ej2.pdb,CCCCNC(=O)[C@H](C)C[C@H](O)[C@@H]2Cc1cccc(c1)O...
152,/tmp/pdb/6ej2.pdb,CCCCNC(=O)[C@H](C)C[C@H](O)[C@@H]2Cc3cc(Cc1ccc...


In [ ]:
# clear out old results if running multiple times -- hopefully they have been downloaded already
!rm -rf /content/DiffDock/results

## Install prerequisites

In [ ]:
!pip install ipython-autotime --quiet
%load_ext autotime

     |████████████████████████████████| 1.6 MB 4.5 MB/s 
time: 398 µs (started: 2022-12-20 08:54:37 +00:00)


In [ ]:
#TODO clean this up and use the dependency in the SIRT3 project
if not os.path.exists("/content/DiffDock"):
    %cd /content
    !git clone https://github.com/gcorso/DiffDock.git
    %cd /content/DiffDock
    !git checkout a6c5275 # remove/update for more up to date code

/content
Cloning into 'DiffDock'...
remote: Enumerating objects: 211, done.
remote: Counting objects: 100% (92/92), done.
remote: Compressing objects: 100% (32/32), done.
remote: Total 211 (delta 73), reused 64 (delta 60), pack-reused 119
Receiving objects: 100% (211/211), 153.23 MiB | 59.87 MiB/s, done.
Resolving deltas: 100% (105/105), done.
/content/DiffDock
Note: checking out 'a6c5275'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  git checkout -b <new-branch-name>

HEAD is now at a6c5275 remove debugging raise in the inference file
time: 5.05 s (started: 2022-12-20 08:54:37 +00:00)


In [ ]:
try:
    import biopandas
    import torch
except:
    !pip install pyg==0.7.1 --quiet
    !pip install pyyaml==6.0 --quiet
    !pip install scipy==1.7.3 --quiet
    !pip install networkx==2.6.3 --quiet
    !pip install biopython==1.79 --quiet
    !pip install rdkit-pypi==2022.03.5 --quiet
    !pip install e3nn==0.5.0 --quiet
    !pip install spyrmsd==0.5.2 --quiet
    !pip install pandas==1.3.5 --quiet
    !pip install biopandas==0.4.1 --quiet
    !pip install torch==1.12.1+cu113 --quiet

     |████████████████████████████████| 65 kB 2.4 MB/s 
     |████████████████████████████████| 1.9 MB 4.7 MB/s 
     |████████████████████████████████| 2.7 MB 4.6 MB/s 
     |████████████████████████████████| 37.0 MB 1.2 MB/s 
     |████████████████████████████████| 117 kB 5.0 MB/s 
     |████████████████████████████████| 878 kB 4.9 MB/s 
ERROR: Could not find a version that satisfies the requirement torch==1.12.1+cu113 (from versions: 1.4.0, 1.5.0, 1.5.1, 1.6.0, 1.7.0, 1.7.1, 1.8.0, 1.8.1, 1.9.0, 1.9.1, 1.10.0, 1.10.1, 1.10.2, 1.11.0, 1.12.0, 1.12.1, 1.13.0, 1.13.1)
ERROR: No matching distribution found for torch==1.12.1+cu113
time: 42.1 s (started: 2022-12-20 08:54:42 +00:00)


In [ ]:
import torch

try:
    import torch_geometric
except ModuleNotFoundError:
    !pip uninstall torch-scatter torch-sparse torch-geometric torch-cluster  --y
    !pip install torch-scatter -f https://data.pyg.org/whl/torch-{torch.__version__}.html --quiet
    !pip install torch-sparse -f https://data.pyg.org/whl/torch-{torch.__version__}.html --quiet
    !pip install torch-cluster -f https://data.pyg.org/whl/torch-{torch.__version__}.html --quiet
    !pip install git+https://github.com/pyg-team/pytorch_geometric.git  --quiet # no version for some reason??

     |████████████████████████████████| 9.4 MB 4.9 MB/s 
     |████████████████████████████████| 4.6 MB 5.0 MB/s 
     |████████████████████████████████| 3.2 MB 4.7 MB/s 
     |████████████████████████████████| 512 kB 4.6 MB/s 
     |████████████████████████████████| 280 kB 91.8 MB/s 
time: 22.6 s (started: 2022-12-20 08:55:24 +00:00)


## Install ESM and prepare PDB file for ESM

In [ ]:
if not os.path.exists("/content/DiffDock/esm"):
    %cd /content/DiffDock
    !git clone https://github.com/facebookresearch/esm
    %cd /content/DiffDock/esm
    !git checkout ca8a710 # remove/update for more up to date code
    !sudo pip install -e .
    %cd /content/DiffDock

/content/DiffDock
Cloning into 'esm'...
remote: Enumerating objects: 879, done.
remote: Counting objects: 100% (267/267), done.
remote: Compressing objects: 100% (131/131), done.
remote: Total 879 (delta 157), reused 229 (delta 132), pack-reused 612
Receiving objects: 100% (879/879), 10.76 MiB | 22.35 MiB/s, done.
Resolving deltas: 100% (516/516), done.
/content/DiffDock/esm
Note: checking out 'ca8a710'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  git checkout -b <new-branch-name>

HEAD is now at ca8a710 Update version.py (#310)
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/DiffDock/esm
  

In [ ]:
%cd /content/DiffDock
!python datasets/esm_embedding_preparation.py --protein_ligand_csv /sirt3/data/processed/affinity_ranking/SIRT3/input_protein_ligand.csv --out_file /sirt3/data/processed/affinity_ranking/SIRT3/prepared_for_esm.fasta 

/content/DiffDock
  0% 0/1 [00:00<?, ?it/s]/usr/local/lib/python3.8/dist-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'END' at line 9575
  warnings.warn(
100% 1/1 [00:00<00:00,  9.61it/s]
time: 717 ms (started: 2022-12-20 08:55:59 +00:00)


In [ ]:
%cd /content/DiffDock
%env HOME=esm/model_weights
%env PYTHONPATH=$PYTHONPATH:/content/DiffDock/esm
!python /content/DiffDock/esm/scripts/extract.py esm2_t33_650M_UR50D /sirt3/data/processed/affinity_ranking/SIRT3/prepared_for_esm.fasta /sirt3/data/processed/affinity_ranking/SIRT3/esm2_output --repr_layers 33 --include per_tok --truncation_seq_length 30000

/content/DiffDock
env: HOME=esm/model_weights
env: PYTHONPATH=$PYTHONPATH:/content/DiffDock/esm
Downloading: "https://dl.fbaipublicfiles.com/fair-esm/models/esm2_t33_650M_UR50D.pt" to esm/model_weights/.cache/torch/hub/checkpoints/esm2_t33_650M_UR50D.pt
Downloading: "https://dl.fbaipublicfiles.com/fair-esm/regression/esm2_t33_650M_UR50D-contact-regression.pt" to esm/model_weights/.cache/torch/hub/checkpoints/esm2_t33_650M_UR50D-contact-regression.pt
Transferred model to GPU
Read /sirt3/data/processed/affinity_ranking/SIRT3/prepared_for_esm.fasta with 1 sequences
Processing 1 of 1 batches (1 sequences)
time: 1min 12s (started: 2022-12-20 08:55:59 +00:00)


## Run DiffDock

In [ ]:
%cd /content/DiffDock
!python -m inference --protein_ligand_csv /sirt3/data/processed/affinity_ranking/SIRT3/input_protein_ligand.csv --esm_embeddings_path /sirt3/data/processed/affinity_ranking/SIRT3/esm2_output --out_dir /sirt3/data/processed/affinity_ranking/SIRT3/ --inference_steps 20 --samples_per_complex 40 --batch_size 32

# Post-process and download results

In [ ]:
%cd /content/DiffDock
!wget https://sourceforge.net/projects/smina/files/smina.static/download -O smina && chmod +x smina

/content/DiffDock
--2022-12-20 09:08:48--  https://sourceforge.net/projects/smina/files/smina.static/download
Resolving sourceforge.net (sourceforge.net)... 104.18.11.128, 104.18.10.128, 2606:4700::6812:a80, ...
Connecting to sourceforge.net (sourceforge.net)|104.18.11.128|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://downloads.sourceforge.net/project/smina/smina.static?ts=gAAAAABjoXugQSapI1rbUWnY0CNxn6I0KIVFf-liwzYM5JxVmDKMalFw43ZGTikjn3lTq0D6UQcMDUiokm1jKkpDdtDC9foQ0g%3D%3D&use_mirror=gigenet&r= [following]
--2022-12-20 09:08:48--  https://downloads.sourceforge.net/project/smina/smina.static?ts=gAAAAABjoXugQSapI1rbUWnY0CNxn6I0KIVFf-liwzYM5JxVmDKMalFw43ZGTikjn3lTq0D6UQcMDUiokm1jKkpDdtDC9foQ0g%3D%3D&use_mirror=gigenet&r=
Resolving downloads.sourceforge.net (downloads.sourceforge.net)... 204.68.111.105
Connecting to downloads.sourceforge.net (downloads.sourceforge.net)|204.68.111.105|:443... connected.
HTTP request sent, awaiting response... 302 

In [ ]:
import re
import pandas as pd
from glob import glob
from shlex import quote
from datetime import datetime
from tqdm.auto import tqdm
from google.colab import files

%cd /sirt3/data/processed/affinity_ranking/SIRT3/
results_dirs = glob("./index*")

rows = []
for results_dir in tqdm(results_dirs, desc="runs"):
    results_pdb_file = "/tmp/pdb/" + re.findall("tmp-pdb-(.+\.pdb)", results_dir)[0]
    results_smiles = re.findall("pdb_+(.+)", results_dir)[0]
    results_sdfs = [os.path.join(results_dir, f) for f in os.listdir(results_dir) if "confidence" in f and f.endswith(".sdf")]

    results_pdb_file_no_hetatms = f"{results_pdb_file}_nohet.pdb"
    !grep -v "^HETATM" {results_pdb_file} > {results_pdb_file_no_hetatms}
    !cp {results_pdb_file} .

    for results_sdf in tqdm(results_sdfs, leave=False, desc="files"):
        confidence = re.findall("confidence([\-\.\d]+)\.sdf", results_sdf)[0]

        scored_stdout = !/content/DiffDock/smina --score_only -r "{results_pdb_file_no_hetatms}" -l "{results_sdf}"
        scored_affinity = re.findall("Affinity:\s*([\-\.\d+]+)", '\n'.join(scored_stdout))[0]
        minimized_stdout = !/content/DiffDock/smina --local_only --minimize -r "{results_pdb_file_no_hetatms}" -l "{results_sdf}" --autobox_ligand "{results_sdf}" --autobox_add 2
        minimized_affinity = re.findall("Affinity:\s*([\-\.\d+]+)", '\n'.join(minimized_stdout))[0]

        rows.append((results_pdb_file.split('/')[-1], results_smiles, float(confidence), float(scored_affinity), float(minimized_affinity), results_sdf))

#
# create dataframe, tar file and download
#
df_results = pd.DataFrame(rows, columns=["pdb_file", "smiles", "diffdock_confidence", "smina_scored_affinity", "smina_minimized_affinity", "sdf_file"])
df_results_tsv = "df_diffdock_results.tsv"
df_results.to_csv(df_results_tsv, sep='\t', index=None)

out_pdbs = ' '.join(set(df_results.pdb_file.apply(quote)))
out_sdfs = ' '.join(df_results.sdf_file.apply(quote))

if download_results:
    tarname = f"diffdock_{datetime.now().isoformat()[2:10].replace('-','')}"
    _ = !tar cvf {tarname}.tar --transform 's,^,{tarname}/,' --transform 's,\./,,' {out_pdbs} {out_sdfs} {df_results_tsv}

    files.download(f"{tarname}.tar")

/sirt3/data/processed/affinity_ranking/SIRT3


runs: 0it [00:00, ?it/s]

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

time: 161 ms (started: 2022-12-20 09:08:49 +00:00)


## Compare smina affinities with DiffDock confidences

In [ ]:
df_results.sort_values("diffdock_confidence", ascending=False)

,pdb_file,smiles,diffdock_confidence,smina_scored_affinity,smina_minimized_affinity,sdf_file


time: 6.76 ms (started: 2022-12-20 09:08:49 +00:00)
